In [2]:
import json
import pandas as pd
import os
import collections
import datetime
import time
import re
import random
from collections import defaultdict

In [3]:
filepath = "sample_files\\01_dummy.json"

with open(filepath) as j_obj:
    data = json.load(j_obj)
    print ('A dictionary is saved!')

A dictionary is saved!


# Helper functions

In [4]:
def convert_TimeString(time_obj):
    tme = time_obj.strftime('%H:%M:%S')
    return tme

In [5]:
def convert_TimeTimedelta(time_obj):
    minutes = time_obj.minute
    sec = time_obj.second
    tot_sec = (minutes*60) + sec
    d = datetime.timedelta(seconds = tot_sec)
    return d

In [6]:
def convert_TimeTimedate(tme):
    """since all my inputs are datetime.time, convert them to timedate object for manipulation"""
    try:
        converted_timedate = datetime.datetime.combine(datetime.date(1,1,1),tme[0])
    except:
        converted_timedate = datetime.datetime.combine(datetime.date(1,1,1),tme)
    return converted_timedate

In [7]:
def convert_StrTime(str_tme):
    tme = datetime.datetime.strptime(str_tme,"%H:%M:%S").time()
    return tme

In [8]:
def convert_IntTime(tme):
    """convert an integear to timedate, then convert to timedelta."""
    if tme >= 60:
        minute,sec = divmod(tme,60)
        str_tme = ':'.join(['00',str(minute),str(sec)])
    else:
        str_tme = ':'.join(['00','00',str(tme)])
    dt = datetime.datetime.strptime(str_tme,"%H:%M:%S").time()   
    return dt

In [9]:
""" A Python Class
A simple Python graph class, demonstrating the essential facts and functionalities of graphs.
"""

class Graph(object):

    def __init__(self, graph_dict=None):
        """ initializes a graph object. If no dictionary or None is given,an empty dictionary will be used."""
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict

    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()

    def __generate_edges(self):
        """ A static method generating the edges of the graph "graph". Edges are represented as sets with one 
            (a loop back to the vertex) or two vertices. """
        edges = []
        #value vertex checked for is the key
        for vertex in self.__graph_dict:
            #get the value of graph[vertex]
            #neighbour is the values/edges to each key/vertex
            if isinstance(vertex,int):
                for neighbour in self.__graph_dict[vertex]:
                    if {neighbour, vertex} not in edges:
                        edges.append({vertex, neighbour})
        return edges

    def __str__(self):
        res = "vertices: "
        for k in self.__graph_dict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def find_all_paths(self,start_vertex,end_vertex, path=[]):
        """ find all paths from start_vertex to end_vertex in graph """
        graph = self.__graph_dict 
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return [path]
        if start_vertex not in graph: #check for dict key
            return []
        paths = []
        for vertex in graph[start_vertex]: #loop thru graph dict using key, obtaining the dict value
            if vertex not in path:
                extended_paths = self.find_all_paths(vertex, end_vertex, path) #reloading funct with list with vertices.
                for p in extended_paths: 
                    paths.append(p)
        return paths
    
    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in self.__graph_dict, a key "vertex" with an empty list as a value 
            is added to the dictionary. Otherwise nothing has to be done."""
        if vertex not in self.__graph_dict:
            self.__graph_dict[vertex] = []

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list;between two vertices can be multiple edges! """
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.__graph_dict:
            self.__graph_dict[vertex1].append(vertex2)
        else:
            self.__graph_dict[vertex1] = [vertex2]

   
    def find_isolated_nodes(graph):
        """ returns a list of isolated nodes. """
        isolated = []
        for node in graph:
            if not graph[node]:
                isolated += node
        return isolated   

# Extracting Resource

In [10]:
def extract_Resources(json_file, json_key, lst_name = []):
    for resource in json_file['resources']:
        if json_key == 'release_time':
            tme = resource['release_time']
            found_tme = re.findall(r'(\d+)',tme)
            lst_name.append(int(found_tme[0]))
        else:
            lst_name.append(resource[json_key])
    return lst_name

In [11]:
#create generic object
Resource = collections.namedtuple('Resource',['resource_id','release_time', 'following_allowed'])

class TrainResource():
    r_id,r_time,following = [],[],[]
    extract_Resources(data,'id',r_id)
    extract_Resources(data,'release_time',r_time)
    extract_Resources(data,'following_allowed',following)
    
    combined = list(zip(r_id,r_time,following))
    
    def __init__(self):
        self._resource = [Resource(a,b,c) for a,b,c in self.combined]
        
    def __len__(self):
        return len(self._resource)

    def __getitem__(self, position):
        return self._resource[position]

In [12]:
resource = TrainResource()

# Extracting Connections

In [13]:
def extract_ConnectID(json_file,lst_name=[]):
    for si in data['service_intentions']:
        for sect_req in si['section_requirements']:
            if (sect_req['connections']):
                lst_name.append(si['id'])
    return lst_name

In [14]:
def extract_ConnectSI(json_file,lst_name=[]):
    for si in data['service_intentions']:
        for sect_req in si['section_requirements']:
            if (sect_req['connections']):
                for connection in sect_req['connections']:
                    lst_name.append(connection['onto_service_intention'])
    return lst_name

In [15]:
def extract_ConnectSM(json_file,lst_name=[]):
    for si in data['service_intentions']:
        for sect_req in si['section_requirements']:
            if (sect_req['connections']):
                for connection in sect_req['connections']:
                    lst_name.append(connection['onto_section_marker'])
    return lst_name

In [16]:
def extract_ConnectTime(json_file,lst_name=[]):
    """To extract connections time"""
    for si in data['service_intentions']:
        for sect_req in si['section_requirements']:
            if (sect_req['connections']):
                for connection in sect_req['connections']:
                    time = connection['min_connection_time']
                    tme = re.findall(r'\d?\d[MS]',time)
                    if (len(tme) == 1) & (tme[0][-1] == 'S'):
                        str_tme = ':'.join(['00','00',tme[0][:-1]])
                    elif (len(tme) == 1) & (tme[0][-1] == 'M'):
                        str_tme = ':'.join(['00',tme[0][:-1],'00'])
                    else:
                        str_tme = ':'.join(['00',tme[0][:-1],tme[1][:-1]])
                    lst_name.append(datetime.datetime.strptime(str_tme,"%H:%M:%S").time())
    return lst_name

In [17]:
Connections = collections.namedtuple('Connections',['route_id','on_si','on_sm','min_connect_time'])

class Connects:
    route_id, on_si,on_sm,connect_time = [],[],[],[]
    extract_ConnectID(data,route_id)
    extract_ConnectSI(data,on_si)
    extract_ConnectSM(data,on_sm)
    extract_ConnectTime(data,connect_time)
    
    combined = list(zip(route_id,on_si,on_sm,connect_time))
    
    def __init__(self):
        self._connect = [Connections(a,b,c,d) for a,b,c,d in self.combined]
        
    def __len__(self):
        return len(self._connect)

    def __getitem__(self, position):
        return self._connect[position]

In [18]:
connections = Connects()

# Train Routes


In [19]:
def extract_AltMarkerEntry(json_file,lst_name=[]):
    """To extract alternative marker"""
    for route in json_file['routes']:
        for id in route['route_paths']:
            for route_section in id['route_sections']:
                try:
                    lst_name.append(route_section['route_alternative_marker_at_entry'][0])
                except:
                    lst_name.append(0)
    return lst_name

In [20]:
def extract_AltMarkerExit(json_file,lst_name=[]):
    """To extract alternative marker"""
    for route in data['routes']:
        for id in route['route_paths']:
            for route_section in id['route_sections']:
                try:
                    lst_name.append(route_section['route_alternative_marker_at_exit'][0])
                except:
                    lst_name.append(0)
    return lst_name

In [21]:
def extract_RSData(json_file,json_key,lst_name = []):
    """3rd loop, sequence_number/penalty"""
    for route in json_file['routes']:
        for id in route['route_paths']:
            for route_section in (id['route_sections']):
                if json_key == 'penalty':
                    txt_to_check = route_section['penalty']
                    if txt_to_check is None:
                        lst_name.append(0)
                    else:
                        lst_name.append(route_section['penalty'])
                else:
                    lst_name.append(route_section[json_key])
    return lst_name

In [22]:
def extract_SectionMarker(json_file,lst_name=[]):
    """3rd loop, section marker"""
    for route in json_file['routes']:
        for id in route['route_paths']:
            for section_marker in (id['route_sections']):
                try:
                    lst_name.append(section_marker['section_marker'][0])
                except:
                    lst_name.append(0)
    return lst_name

In [23]:
def extract_RoutePathID(json_file,lst_name = []):
    '''extract route paths id of recommended routes(2nd depth)'''
    for route in json_file['routes']:
        for id in (route['route_paths']):
            length = len(id['route_sections'])
            y = 1
            while y <= length:
                lst_name.append(id['id'])
                y +=1
    return lst_name

In [24]:
def extract_TrainIDRoutesAltMarker(json_file,lst_name=[]):
    """Extract ID of suggested routes of trains for route alternative marker"""
    for index,route in enumerate(json_file['routes']):
        train_id = route['id']
        total = 0
        for id in route['route_paths']:
            num_lst = []
            for index,sm in enumerate(id['route_sections'],1):
                num_lst.append(index)
                num = num_lst[-1]
            total = total + num
        y = 1
        while y <= total:
            lst_name.extend([train_id] * 2)
            y += 1
    return lst_name

In [25]:
def extract_TrainIDRoutes(json_file,lst_name=[]):
    """Extract ID of suggested routes of trains"""
    for index,route in enumerate(json_file['routes']):
        train_id = route['id']
        total = 0
        for id in route['route_paths']:
            num_lst = []
            for index,sm in enumerate(id['route_sections'],1):
                num_lst.append(index)
                num = num_lst[-1]
            total = total + num
        y = 1
        while y <= total:
            lst_name.append(train_id)
            y += 1
    return lst_name

In [26]:
def extract_MinRunTime(json_file,lst_name=[]):
    """Min running time for each route section"""
    for route in data['routes']:
        for time in route['route_paths']:
            for run_time in (time['route_sections']):
                time = run_time['minimum_running_time']
                tme = re.findall(r'\d?\d[MS]',time)
                if (len(tme) == 1) & (tme[0][-1] == 'S'):
                    str_tme = ':'.join(['00','00',tme[0][:-1]])
                elif (len(tme) == 1) & (tme[0][-1] == 'M'):
                    str_tme = ':'.join(['00',tme[0][:-1],'00'])
                else:
                    str_tme = ':'.join(['00',tme[0][:-1],tme[1][:-1]])
                lst_name.append(datetime.datetime.strptime(str_tme,"%H:%M:%S").time())
    return lst_name

In [27]:
def extract_ResourceOccupation(json_file,lst_name=[]):
    for route in json_file['routes']:
        for id in route['route_paths']:
            for section_marker in (id['route_sections']):
                resource_lst = []
                for resource in section_marker['resource_occupations']:
                    r = resource['resource']
                    resource_lst.append(r)
                lst_name.append(resource_lst)

In [28]:
#create generic object
Route = collections.namedtuple('Route',['train_id','route_path_id','seq_num','sect_marker','ram_entry','ram_exit',
                                        'min_run_time','penalty','resource'])

class TrainRoutes:
    #get data from json file using customized functions
    sequence_numbers,section_markers,route_path_ids,train_ids,penalty,min_run_time= [],[],[],[],[],[]
    ram_entry,ram_exit,resource = [],[],[]
    extract_RSData(data,'sequence_number',sequence_numbers)
    extract_SectionMarker(data,section_markers)
    extract_RoutePathID(data,route_path_ids)
    extract_TrainIDRoutes(data,train_ids)
    extract_AltMarkerEntry(data,ram_entry)
    extract_AltMarkerExit(data,ram_exit)
    extract_RSData(data,'penalty',penalty)
    extract_MinRunTime(data,min_run_time)
    extract_ResourceOccupation(data,resource)
    
    #
    combined = list(zip(train_ids,route_path_ids,sequence_numbers,section_markers,ram_entry,ram_exit,min_run_time,penalty,resource))
    
    def __init__(self):
        self._routes = [Route(a,b,c,d,e,f,g,h,i) for a,b,c,d,e,f,g,h,i in self.combined]
        
    def __len__(self):
        return len(self._routes)

    def __getitem__(self, position):
        return self._routes[position]

In [29]:
train_route = TrainRoutes()

# Service Intentions (trains to be scheduled)

In [30]:
def extract_ID(json_key,json_file,lst_name=[]):
    '''Extraction service intention ID or route ID(1st loop)'''
    if json_key == 'service_intentions':
        for si in json_file[json_key]:
            length = len(si['section_requirements'])
            y = 1
            while y <= length:
                lst_name.append(si['id'])
                y +=1
    elif json_key == 'routes':
        for route in json_file[json_key]:
            lst_name.append(route['id'])
    return lst_name

In [31]:
def extract_SIData(json_file,json_key,lst_name=[]):
    """To extract section_requirements of SI (sequence_num,sect_mark,entry_wgt,exit_wgt)"""
    for si in json_file['service_intentions']:
        for sect_req in si['section_requirements']:
            try:
                lst_name.append(sect_req[json_key])
            except:
                lst_name.append(0)
    return lst_name

In [32]:
def extract_Timings(json_file,json_key,lst_name=[]):
    """To extract section_requirements of SI"""
    for si in json_file['service_intentions']:
        for index,sect_req in enumerate(si['section_requirements']):
            #timings = ['entry_earliest','exit_earliest','exit_latest']
            #for time in timings:
            try:
                tme = sect_req[json_key]
                dt = datetime.datetime.strptime(tme,"%H:%M:%S").time()
                lst_name.append(dt)
            except:
                lst_name.append(0)
    return lst_name 

In [33]:
def extract_MinStopTime(json_file,lst_name=[]):
    """To extract Minimum Stopping Time"""
    for si in json_file['service_intentions']:
        for sect_req in si['section_requirements']:
            try:
                time = sect_req['min_stopping_time']
                tme = re.findall(r'\d?\d[MS]',time)
                if (len(tme) == 1) & (tme[0][-1] == 'S'):
                    str_tme = ':'.join(['00','00',tme[0][:-1]])
                elif (len(tme) == 1) & (tme[0][-1] == 'M'):
                    str_tme = ':'.join(['00',tme[0][:-1],'00'])
                else:
                    str_tme = ':'.join(['00',tme[0][:-1],tme[1][:-1]])
                lst_name.append(datetime.datetime.strptime(str_tme,"%H:%M:%S").time())
            except:
                lst_name.append(0)
    return lst_name

In [200]:
txt = 'PT10S'
tme = re.findall(r'\d?\d[MS]',txt)
if (len(tme) == 1) & (tme[0][-1] == 'S'):
    str_tme = ':'.join(['00','00',tme[0][:-1]])
elif (len(tme) == 1) & (tme[0][-1] == 'M'):
    str_tme = ':'.join(['00',tme[0][:-1],'00'])
else:
    str_tme = ':'.join(['00',tme[0][:-1],tme[1][:-1]])
datetime.datetime.strptime(str_tme,"%H:%M:%S").time()

datetime.time(0, 0, 10)

In [34]:
#create generic object
Service_Intention = collections.namedtuple('SI',['si_id','seq_num','sect_marker','entry_earliest','entry_latest','exit_earliest',
                                                 'exit_latest','stop_time','entry_wgt','exit_wgt'])

class SI_Routes:
    #get data from json file using customized functions
    si_sequence_numbers,si_section_markers,si_id,ent_early,ent_late = [],[],[],[],[]
    ext_early,ext_late = [],[]
    stop_time,entry_delay_weight,exit_delay_weight = [],[],[]
    extract_SIData(data,'sequence_number',si_sequence_numbers)
    extract_SIData(data,'section_marker',si_section_markers)
    extract_ID('service_intentions',data,si_id)
    extract_Timings(data,'entry_earliest',ent_early)
    extract_Timings(data,'entry_latest',ent_late)
    extract_Timings(data,'exit_earliest',ext_early)
    extract_Timings(data,'exit_latest',ext_late)
    extract_MinStopTime(data,stop_time)
    extract_SIData(data,'entry_delay_weight',entry_delay_weight)
    extract_SIData(data,'exit_delay_weight',exit_delay_weight)
    
    
    combined = list(zip(si_id,si_sequence_numbers,si_section_markers,ent_early,ent_late,ext_early,ext_late,
                        stop_time,entry_delay_weight,exit_delay_weight))
    
    def __init__(self):
        self._routes = [Service_Intention(a,b,c,d,e,f,g,h,i,j) for a,b,c,d,e,f,g,h,i,j in self.combined]
        
    def __len__(self):
        return len(self._routes)

    def __getitem__(self, position):
        return self._routes[position]

In [35]:
si = SI_Routes()

# Starts Here

In [36]:
route_alternative = []
for r in train_route:
    to_append = r.train_id,r.route_path_id,r.sect_marker,r.seq_num,(r.ram_entry,r.ram_exit) #r.penalty
    route_alternative.append(to_append)
print ('Number of train routes: ',len(route_alternative))

Number of train routes:  318


In [37]:
#create a list with the indices of the train_id to extract the rows attached to a train_id
lst = [0]
for idx,r in enumerate(train_route[:-1]):
    if r.train_id == train_route.train_ids[idx+1]:
        pass
    else:
        lst.append(idx+1)
lst.append(idx+2)
#output is [0,14,28], which tells me train_id '1' belongs to row (0 to 14), train_id '2' belongs to (14 to 28)

train_indexes = []
for idx,l in enumerate(lst[:-1]):
    paired = (l,lst[idx+1])
    train_indexes.append(paired)

#create a graph for each train route
all_graphs = []
for idx in train_indexes:
    #extract only route for the same train_id
    route = route_alternative[idx[0]:idx[1]]
    
    graph = defaultdict(list)
    for index,tup in enumerate(route[:-1]):
        graph['train_id'] = route[0][0]
        if tup[3] < route[index+1][3]:
        #checks current exit against next entry
            if tup[4][1] == route[index+1][4][0]:
                exit = tup[3]
                nxt_entry = route[index+1][3]    
                graph[exit] = [nxt_entry]
    missing_graph = defaultdict(list)
    #obtain all exit values
    exit_lst = [{x[3]:x[4][1]} for x in route if isinstance(x[4][1],str)]
    #iterate over them to get entry values and sequence
    for ra in route:
        for row in exit_lst:
            for k,v in row.items():
                #compare entry values to defined exit values
                if (ra[4][0] == v) & (ra[3] > k):
                    #obtain exit values
                    seq = ra[3]
                    missing_graph[k] = [seq]
    
    #update graph with missing_graph entries
    for k, v in missing_graph.items():
        if graph.get(k,[]) == v:
            pass
        else:
            graph[k] = graph.get(k, []) + v
    #all_graphs['train_id'] = []
    all_graphs.append(graph)

In [38]:
all_paths = []
path_dict = defaultdict(list) 
for graph in all_graphs:
    t_id = graph['train_id']
    path_dict['train_id'] = t_id
    section_marker = [s.sect_marker for s in si if s.si_id == t_id]
    start_vertex = [r.seq_num for r in train_route if (r.sect_marker == section_marker[0]) & (r.train_id == t_id)]
    end_vertex = [r.seq_num for r in train_route if (r.sect_marker == section_marker[-1]) & (r.train_id == t_id)]
       
    g = Graph(graph)
    vertices = [[x,y] for x in start_vertex for y in end_vertex]
    for vertex in vertices:
        start,end = vertex[0],vertex[1]
        path = g.find_all_paths(start,end)
        path_dict['path'] = path
        all_penalty,penalty = [],[]
        flatten_path = [seq_num for p in path for seq_num in p]
        for seq_num in flatten_path:
            seq_penalty = [r.penalty for r in train_route if (r.seq_num == seq_num) & (r.train_id == t_id)][0]
            penalty.append(seq_penalty)
        sum_penalty = sum(penalty)
        all_penalty.append(sum_penalty)
        path_dict['penalty'] = all_penalty
        all_paths.append(path_dict.copy())

print ('There are {} graphs'.format(len(all_graphs)))
print ('There are {} suggested paths'.format(len(all_paths)))

There are 4 graphs
There are 4 suggested paths


In [42]:
#identify how many service intentions(trains) to schedule
lst = []
for s_i in si:
    lst.append(s_i.si_id)
    trains = set(lst)
print ('There are {} trains to be scheduled'.format(len(trains)))
print ('The IDs are: ',trains)

There are 4 trains to be scheduled
The IDs are:  {18825, 20423, 20425, 18823}


In [43]:
#identify how many vertices (aka sequence number) there are
lst = []
for r in train_route:
    lst.append(r.seq_num)
    vertex = set(lst)
print ('There are {} unique sequences and they are {}'.format(len(vertex),vertex))

There are 93 unique sequences and they are {1, 5, 10, 15, 20, 25, 30, 35, 37, 40, 45, 50, 52, 55, 60, 65, 70, 72, 75, 80, 85, 90, 95, 100, 102, 105, 110, 115, 120, 122, 125, 130, 135, 140, 142, 145, 150, 155, 160, 162, 165, 170, 172, 175, 177, 180, 185, 187, 190, 195, 200, 205, 210, 212, 215, 220, 225, 230, 235, 237, 240, 245, 250, 255, 257, 260, 262, 265, 270, 275, 277, 280, 285, 287, 290, 295, 300, 305, 403, 404, 405, 406, 407, 408, 409, 410, 411, 500, 501, 502, 503, 504, 505}


# Building the list

In [78]:
solution = {'problem_instance_label': data['label'],
           'problem_instance_hash': data['hash'],
           'hash':data['hash'],
            'train_runs':[]
           }

train_run_sections = []
for train in trains:
    t_id = train
    path_lst = random.choice([path['path'] for path in all_paths if path['train_id'] == t_id])
    if len(path_lst) > 1:
        c_path = random.choice(path_lst)
    else:
        c_path = path_lst[0]
    
    marker_seq = sorted([(r.sect_marker,r.seq_num) for r in train_route for seq in c_path if (r.seq_num == seq) & (r.train_id == t_id)],
                        key = lambda x:x[1])
    
    extract_route_section_id = sorted([(r.train_id,r.seq_num) for r in train_route for seq in c_path if (r.seq_num == seq) & (r.train_id == t_id)],
                                      key = lambda x:x[1])
    route_section_id = ['#'.join([str(t[0]),str(t[1])]) for t in extract_route_section_id]
    
    
    #train_runs_lst = [{'service_intention_id':t_id,'train_run_sections':[]}]
    train_runs_lst = {'service_intention_id':t_id}
    lst = []
    for index,path in enumerate(marker_seq,1):
        section_requirement,sequence_number = path[0],path[1]
        lst.append({'section_requirement':section_requirement,'sequence_number':index,
                'entry_time':[(s_i.entry_earliest or s_i.entry_latest) for s_i in si if (s_i.sect_marker == section_requirement) & (s_i.si_id == t_id)],
                'exit_time':[(s_i.exit_earliest or s_i.exit_latest) for s_i in si if (s_i.sect_marker == section_requirement) & (s_i.si_id == t_id)],
                'route':t_id,
               'route_section_id':route_section_id[index-1],
                'route_path':[r.route_path_id for r in train_route if (r.seq_num == sequence_number) & (r.train_id == t_id)][0]
               })
    
    train_runs_lst['train_run_sections'] = lst
    train_run_sections.append(train_runs_lst)
    print ('{} scheduled and completed'.format(t_id))
solution['train_runs'] = train_run_sections
print ('there are {} trains scheduled'.format(len(solution)))    
    
 #train_runs_lst['train_run_sections'] = lst
    #train_run_sections.append(train_runs_lst)
    

#solution['train_runs'] = train_run_sections


18825 scheduled and completed
20423 scheduled and completed
20425 scheduled and completed
18823 scheduled and completed
there are 4 trains scheduled


In [81]:
for lst in solution['train_runs']:
    last = None
    for row in lst['train_run_sections']:
        if last is not None:
            row["entry_time"] = last["exit_time"]
    
        t_id = row['route']
        section_requirement = row['section_requirement']
        route_section_id = row['route_section_id']
        seq_n = int(re.findall(r'\d+',route_section_id)[1])
        converted_entry_time = convert_TimeTimedate(row['entry_time'])
        exit_time = (converted_entry_time + delta).time()
        #check if 'exit_time' is [0] or []
        if (0 in row['exit_time']) | (not row['exit_time']) :
            #update row(current) with exit time and state of last changed to row.
            row["exit_time"] = [exit_time]
        last = row
        

       
        if [s.sect_marker for s in si if (s.si_id == t_id) & (s.sect_marker == section_requirement)]:
            #get stop time and min run time
            try:
                stop_time = convert_TimeTimedelta([s.stop_time for s in si if (s.sect_marker == section_requirement) 
                                             & (s.si_id == t_id)][0])
            except:
                stop_time = datetime.timedelta(seconds=0)
            min_run_time = convert_TimeTimedelta([t.min_run_time for t in train_route if (t.train_id == t_id) 
                                                  & (t.seq_num == seq_n) & (t.sect_marker == section_requirement)][0])
            tot_run_time = (stop_time + min_run_time)
        else:
            tot_run_time = convert_TimeTimedelta([t.min_run_time for t in train_route if (t.train_id == t_id) 
                                                  & (t.seq_num == seq_n) & (t.sect_marker == section_requirement)][0])
        exit_time = (converted_entry_time + tot_run_time).time()
        if (row['exit_time'] == []) | (row['exit_time'] == [0]):
            row['exit_time'] = [exit_time]
        else: #check the existing exit value
            if (exit_time >= row['exit_time'][0]):
                row['exit_time'] = [exit_time]


In [82]:
solution

{'hash': 759370455,
 'problem_instance_hash': 759370455,
 'problem_instance_label': '01_dummy',
 'train_runs': [{'service_intention_id': 18825,
   'train_run_sections': [{'entry_time': [datetime.time(7, 5)],
     'exit_time': [datetime.time(7, 7)],
     'route': 18825,
     'route_path': 'standard',
     'route_section_id': '18825#1',
     'section_requirement': 'ZLOE_Halt',
     'sequence_number': 1},
    {'entry_time': [datetime.time(7, 7)],
     'exit_time': [datetime.time(7, 7, 32)],
     'route': 18825,
     'route_path': 'standard',
     'route_section_id': '18825#5',
     'section_requirement': 'ZLOE',
     'sequence_number': 2},
    {'entry_time': [datetime.time(7, 7, 32)],
     'exit_time': [datetime.time(7, 8, 2)],
     'route': 18825,
     'route_path': 'standard',
     'route_section_id': '18825#10',
     'section_requirement': 'ZUET40',
     'sequence_number': 3},
    {'entry_time': [datetime.time(7, 8, 2)],
     'exit_time': [datetime.time(7, 8, 32)],
     'route': 18825,

# Convert file to json

In [83]:
for train_run in solution['train_runs']:
    for row in train_run['train_run_sections']:
        row['entry_time'] = convert_TimeString(row['entry_time'][0])
        row['exit_time'] = convert_TimeString(row['exit_time'][0])
        sm = row['section_requirement']
        if sm in [s.sect_marker for s in si if (s.si_id == row['route'])]:
            pass
        else:
            row['section_requirement'] = None

In [84]:
with open("01_mysolution_new.json","w") as f:
    json.dump(solution,f,indent = 4)

In [85]:
%run validate_solution.py

upload finished with status 200
validation finished with status 200

There are 10 warnings and 2 errors

the solution has 2 errors. It will not be accepted as a feasible solution. See the error messages for details.

Errors:
- Entry time 07:30:24 after einMax 07:25 (+ max. Bandabweichung PT5M) for Train run sections with route_section_id "20423#295" and Section Marker "ZG_Halt" in service_intention "20423"
- Entry time 08:00:12 after einMax 07:55 (+ max. Bandabweichung PT5M) for Train run sections with route_section_id "20425#295" and Section Marker "ZG_Halt" in service_intention "20425"

Warnings:
- Solution with VP-Label "01_dummy" and problem_instance_hash "759370455" has a wrong Hash! Hash: 759370455, expected: -969145936
- Entry time 07:52:15 after einMax 07:48 for Train run sections with route_section_id "18825#305" and Section Marker "PF_Halt" in service_intention "18825"
- Entry time 07:40:19 after einMax 07:38 for Train run sections with route_section_id "18825#237" and Sectio